In [37]:
import http.client
import mimetypes
from codecs import encode
from bs4 import BeautifulSoup
import chardet
import csv

def create_multipart_data():
    boundary = 'wL36Yn8afVp8Ag7AmP8qZ0SA4n1v9T'
    dataList = []
    parts = [
        ('c', ''),
        ('codigoDesaparecido', ''),
        ('menor', 'false'),
        ('nome', ''),
        ('municipio', ''),
        ('_', '')
    ]
    for key, value in parts:
        dataList.append(encode('--' + boundary))
        dataList.append(encode(f'Content-Disposition: form-data; name={key};'))
        dataList.append(encode('Content-Type: {}'.format('text/plain')))
        dataList.append(encode(''))
        dataList.append(encode(value))
    dataList.append(encode('--'+boundary+'--'))
    dataList.append(encode(''))
    return b'\r\n'.join(dataList), boundary

def send_request(payload, boundary, index):
    headers = {
        'Content-type': f'multipart/form-data; boundary={boundary}'
    }
    conn = http.client.HTTPSConnection("www.desaparecidos.pr.gov.br")
    try:
        conn.request("POST", f"/desaparecidos/desaparecidos.do?action=pesquisar&indice={index}&dummy=521191014869541100_1714670260366", payload, headers)
        res = conn.getresponse()
        data = res.read()
        encoding = chardet.detect(data)['encoding']
        return data.decode(encoding)
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        conn.close()

def process_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    div_texts = []
    divs = soup.find_all('div', class_='col-md-9')
    pessoas = []
    for div in divs:
        div_texts.append(div.text.strip().split("/n"))

    for div_row in div_texts:
        pessoas.append(div_row[0].split("\n"))

    for pessoa in pessoas:
        print(pessoa, "\n")

    return pessoas

def save_to_csv(data):
    with open('Desaparecidos Polícia Civil PR - Adultos.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['nome', 'resto'])
        for item in data:
            writer.writerow(item)

def main():
    payload, boundary = create_multipart_data()
    all_texts = []
    for index in range(1, 439):
        html = send_request(payload, boundary, index)
        if html:
            div_texts = process_html(html)
            all_texts.extend(div_texts)
    save_to_csv(all_texts)  # Salva todos os dados ao final do processo

main()

['RAFAEL MÉDICE', 'Desaparecido desde:02/05/2024', 'Município: CURITIBA', 'Idade (desaparecimento): 66  anos', 'Idade (atual): 66 anos', 'Trajava na ocasião: NAO SABE INFORMAR', 'Cor dos cabelos: BRANCOS', 'Olhos: CASTANHOS CLAROS'] 

['ADELINO TREVISAN', 'Desaparecido desde:02/05/2024', 'Município: FRANCISCO BELTRAO', 'Idade (desaparecimento): 81  anos', 'Idade (atual): 81 anos', 'Trajava na ocasião: PROVAVELMENTE COM TENIS E CALÇA JEANS ESCURA', 'Cor dos cabelos: BRANCOS', 'Olhos: CASTANHOS CLAROS'] 

['ANA CRISTINA DOS SANTOS', 'Desaparecido desde:01/05/2024', 'Município: SAO JOSE DOS PINHAIS', 'Idade (desaparecimento): 26  anos', 'Idade (atual): 26 anos', 'Trajava na ocasião: SAIA ESCURA E BLUSA ROSA', 'Cor dos cabelos: CASTANHOS ESCUROS', 'Olhos: CASTANHOS AZULADOS'] 

['ADALBERTO SOUZA DE OLIVEIRA', 'Desaparecido desde:01/05/2024', 'Município: SARANDI', 'Idade (desaparecimento): 33  anos', 'Idade (atual): 33 anos', 'Trajava na ocasião: NÃO SABE INFORMAR .', 'Cor dos cabelos: NÃO 